# Team Members:
- Amirhossein Mobayen
- Mohamed amine Filali
- Nikita Chistyakov
- Rym Mehdi
- Skander Ben brik

## Topic:
AI Chess Master

# Objective:
BuildaVisionAIwhichunderstandsapositionbylookingattheboard!

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Step 1: Dataset Preparation


In [ ]:
# Download the dataset from Kaggle and extract the relevant files.
# URL: https://www.kaggle.com/koryakinp/chess-positions

# Define the input image size for the model
input_shape = (224, 224, 3)

# Define the path to the dataset directory
dataset_dir = 'path/to/dataset'

# Step 2: Data Preprocessing

In [ ]:
# Load and preprocess a chessboard image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=input_shape[:2])
    img_array = img_to_array(img)
    img_array = img_array.reshape((1,) + img_array.shape)
    img_array = img_array.astype('float32')
    img_array /= 255.0
    return img_array

In [ ]:
# Iterate through the dataset directory to preprocess all images
image_paths = []
labels = []

# Iterate through each subdirectory (class) in the dataset directory
for subdirectory in os.listdir(dataset_dir):
    subdirectory_path = os.path.join(dataset_dir, subdirectory)
    if os.path.isdir(subdirectory_path):
        # Iterate through each image file in the subdirectory
        for image_file in os.listdir(subdirectory_path):
            image_path = os.path.join(subdirectory_path, image_file)
            image_paths.append(image_path)
            labels.append(subdirectory)  # Assuming subdirectory name is the label

# Convert labels to one-hot encoding
num_classes = len(set(labels))
labels = tf.keras.utils.to_categorical(labels, num_classes=num_classes)

# Split the dataset into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# Preprocess the training images
train_images_preprocessed = []
for image_path in train_images:
    preprocessed_image = preprocess_image(image_path)
    train_images_preprocessed.append(preprocessed_image)
train_images_preprocessed = np.array(train_images_preprocessed)

# Preprocess the testing images
test_images_preprocessed = []
for image_path in test_images:
    preprocessed_image = preprocess_image(image_path)
    test_images_preprocessed.append(preprocessed_image)
test_images_preprocessed = np.array(test_images_preprocessed)

# Step 3: Model Selection and Architecture

In [ ]:
# Load the VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Add a global average pooling layer and a fully connected layer on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)


# Step 4: Model Training

In [ ]:
# Compile the model with an appropriate loss function and optimizer
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images_preprocessed, train_labels, batch_size=32, epochs=10, validation_data=(test_images_preprocessed, test_labels))


# Step 5: Model Evaluation

In [ ]:
# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

# Step 6: Model Deployment

In [ ]:
# Save the trained model weights for future use
model.save('chessboard_model.h5')

# Load the saved model
loaded_model = tf.keras.models.load_model('chessboard_model.h5')

# Preprocess a chessboard image and make predictions
image_path = 'path/to/chessboard_image.jpg'
preprocessed_image = preprocess_image(image_path)
preprocessed_image = np.expand_dims(preprocessed_image, axis=0)
predictions = loaded_model.predict(preprocessed_image)

In [ ]:
# Convert predictions to FEN format
def convert_predictions_to_fen(predictions):
    # Implement your conversion logic here
    return 'FEN position'

In [ ]:
# Convert predictions to FEN format
predicted_position = convert_predictions_to_fen(predictions)

# Print the predicted position
print(predicted_position)